In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook
import statistics

In [3]:
links = pd.read_csv(r'D:\Python3\Machine_learning2\Рекомендательные системы\netology-recsys-master\lecture-1\links.csv')
movies = pd.read_csv(r'D:\Python3\Machine_learning2\Рекомендательные системы\netology-recsys-master\lecture-1\movies.csv')
ratings = pd.read_csv(r'D:\Python3\Machine_learning2\Рекомендательные системы\netology-recsys-master\lecture-1\ratings.csv')
tags = pd.read_csv(r'D:\Python3\Machine_learning2\Рекомендательные системы\netology-recsys-master\lecture-1\tags.csv')

In [4]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [6]:
tags.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [7]:
# соединим таблицу с рейтингами и названиями фильмов
joined_ratings = ratings.join(movies.set_index('movieId'), on='movieId')

In [8]:
joined_ratings.head()

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,964981247,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,964982224,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,964982931,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


#### Считаем, сколько оценок выставил каждый пользователь

In [9]:
rating_pivot_user_rating = ratings.pivot_table(index = 'userId', columns = 'rating', values = 'timestamp', aggfunc = 'count', fill_value = 0)
rating_pivot_user_rating.head()

rating,0.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0
userId,,,,,,,,,,
1,0,1,0,5,0,26,0,76,0,124
2,0,0,0,1,1,4,4,9,4,6
3,20,0,0,1,0,1,1,1,5,10
4,0,23,0,26,0,39,0,64,0,64
5,0,1,0,3,0,17,0,13,0,10


In [10]:
rating_pivot_film_rating = ratings.pivot_table(index = 'movieId', columns = 'rating', values = 'timestamp', aggfunc = 'count', fill_value = 0)
rating_pivot_film_rating.head()

rating,0.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0
movieId,,,,,,,,,,
1,1,0,1,6,8,34,18,82,18,47
2,1,1,2,5,12,28,11,36,7,7
3,1,3,1,2,2,20,5,12,0,6
4,0,1,1,1,0,4,0,0,0,0
5,1,0,2,6,2,25,1,8,1,3


In [11]:
joined_ratings.rating.value_counts().sort_index()

0.5     1370
1.0     2811
1.5     1791
2.0     7551
2.5     5550
3.0    20047
3.5    13136
4.0    26818
4.5     8551
5.0    13211
Name: rating, dtype: int64

### Добавляем значения по пользователям (mean, median, variance)

In [12]:
user_mean_rating = joined_ratings.groupby('userId')['rating'].mean()
user_mean_rating.rename(('user_mean_rating'), inplace=True)

userId
1      4.366379
2      3.948276
3      2.435897
4      3.555556
5      3.636364
         ...   
606    3.657399
607    3.786096
608    3.134176
609    3.270270
610    3.688556
Name: user_mean_rating, Length: 610, dtype: float64

In [13]:
user_median_rating = joined_ratings.groupby('userId')['rating'].median()
user_median_rating.rename(('user_median_rating'), inplace=True)

userId
1      5.0
2      4.0
3      0.5
4      4.0
5      4.0
      ... 
606    4.0
607    4.0
608    3.0
609    3.0
610    3.5
Name: user_median_rating, Length: 610, dtype: float64

In [14]:
joined_ratings = joined_ratings.merge(user_mean_rating, on='userId').merge(user_median_rating, on='userId')
joined_ratings.head()

,userId,movieId,rating,timestamp,title,genres,user_mean_rating,user_median_rating
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,4.366379,5.0
1,1,3,4.0,964981247,Grumpier Old Men (1995),Comedy|Romance,4.366379,5.0
2,1,6,4.0,964982224,Heat (1995),Action|Crime|Thriller,4.366379,5.0
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,4.366379,5.0
4,1,50,5.0,964982931,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,4.366379,5.0


In [15]:
user_list = joined_ratings.userId.unique()
user_list = user_list.tolist()
# user_list

In [16]:
user_variance_list = []
for i in user_list:
    if len(joined_ratings[joined_ratings.userId == i]['rating']) == 1:
        user_variance_list.append(0)
    else:
        user_variance_list.append(statistics.variance(joined_ratings[joined_ratings.userId == i]['rating']))

In [17]:
user_variance = pd.DataFrame(list(zip(user_list, user_variance_list)),

               columns =['userId1', 'user_variance'])

In [18]:
joined_ratings = pd.merge(joined_ratings, user_variance, left_on='userId', right_on='userId1')
joined_ratings = joined_ratings.drop('userId1', axis = 1)

### Добавляем значения по фильмам (mean, median, variance)

In [19]:
film_mean_rating = joined_ratings.groupby('movieId')['rating'].mean()
film_mean_rating.rename(('film_mean_rating'), inplace=True)

movieId
1         3.920930
2         3.431818
3         3.259615
4         2.357143
5         3.071429
            ...   
193581    4.000000
193583    3.500000
193585    3.500000
193587    3.500000
193609    4.000000
Name: film_mean_rating, Length: 9724, dtype: float64

In [20]:
film_median_rating = joined_ratings.groupby('movieId')['rating'].median()
film_median_rating.rename(('film_median_rating'), inplace=True)

movieId
1         4.0
2         3.5
3         3.0
4         3.0
5         3.0
         ... 
193581    4.0
193583    3.5
193585    3.5
193587    3.5
193609    4.0
Name: film_median_rating, Length: 9724, dtype: float64

In [21]:
joined_ratings = joined_ratings.merge(film_mean_rating, on='movieId').merge(film_median_rating, on='movieId')
joined_ratings.head()

,userId,movieId,rating,timestamp,title,genres,user_mean_rating,user_median_rating,user_variance,film_mean_rating,film_median_rating
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,4.366379,5.0,0.640077,3.92093,4.0
1,5,1,4.0,847434962,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.636364,4.0,0.980973,3.92093,4.0
2,7,1,4.5,1106635946,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.230263,3.5,1.767820,3.92093,4.0
3,15,1,2.5,1510577970,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.448148,3.5,1.284605,3.92093,4.0
4,17,1,4.5,1305696483,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,4.209524,4.0,0.258562,3.92093,4.0


In [22]:
film_list = joined_ratings.movieId.unique()
film_list = film_list.tolist()
# film_list

In [23]:
film_variance_list = []
for i in film_list:
    if len(joined_ratings[joined_ratings.movieId == i]['rating']) == 1:
        film_variance_list.append(0)
    else:
        film_variance_list.append(statistics.variance(joined_ratings[joined_ratings.movieId == i]['rating']))

In [24]:
film_variance = pd.DataFrame(list(zip(film_list, film_variance_list)),

               columns =['movieId1', 'movie_variance'])

In [25]:
joined_ratings = pd.merge(joined_ratings, film_variance, left_on='movieId', right_on='movieId1')
joined_ratings = joined_ratings.drop('movieId1', axis = 1)

In [26]:
joined_ratings.head()

,userId,movieId,rating,timestamp,title,genres,user_mean_rating,user_median_rating,user_variance,film_mean_rating,film_median_rating,movie_variance
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,4.366379,5.0,0.640077,3.92093,4.0,0.69699
1,5,1,4.0,847434962,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.636364,4.0,0.980973,3.92093,4.0,0.69699
2,7,1,4.5,1106635946,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.230263,3.5,1.767820,3.92093,4.0,0.69699
3,15,1,2.5,1510577970,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.448148,3.5,1.284605,3.92093,4.0,0.69699
4,17,1,4.5,1305696483,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,4.209524,4.0,0.258562,3.92093,4.0,0.69699


In [27]:
years = []

for title in joined_ratings['title']:
    year_subset = title[-5:-1]
    try: years.append(int(year_subset))
    except: years.append(9999)
        
joined_ratings['movieYear'] = years
print(len(joined_ratings[joined_ratings['movieYear'] == 9999]))

30


In [28]:
X = joined_ratings.drop(['title', 'rating','timestamp','genres'], axis = 1)
X.head()

,userId,movieId,user_mean_rating,user_median_rating,user_variance,film_mean_rating,film_median_rating,movie_variance,movieYear
0,1,1,4.366379,5.0,0.640077,3.92093,4.0,0.69699,1995
1,5,1,3.636364,4.0,0.980973,3.92093,4.0,0.69699,1995
2,7,1,3.230263,3.5,1.767820,3.92093,4.0,0.69699,1995
3,15,1,3.448148,3.5,1.284605,3.92093,4.0,0.69699,1995
4,17,1,4.209524,4.0,0.258562,3.92093,4.0,0.69699,1995


In [29]:
y = joined_ratings.rating

In [30]:
from sklearn.model_selection import train_test_split

rand_state = 10

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=rand_state)

Построим линейную модель:

In [31]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error

model = LinearRegression()

model.fit(X_train, y_train)

train_pred = model.predict(X_train)
test_pred = model.predict(X_test)


train_mae = mean_absolute_error(train_pred, y_train)
test_mae = mean_absolute_error(test_pred, y_test)

print(f'Train MAE: {np.round(train_mae, 4)}')
print(f'Test MAE: {np.round(test_mae, 4)}')


Train MAE: 0.6148
Test MAE: 0.612


In [32]:
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer

In [33]:
# count_vect = CountVectorizer()
# X = count_vect.fit_transform(joined_ratings.genres)
# X.toarray()

In [34]:
new_data = movies.merge(film_median_rating, on='movieId')
new_data.head(10)

,movieId,title,genres,film_median_rating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,4.0
1,2,Jumanji (1995),Adventure|Children|Fantasy,3.5
2,3,Grumpier Old Men (1995),Comedy|Romance,3.0
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,3.0
4,5,Father of the Bride Part II (1995),Comedy,3.0
5,6,Heat (1995),Action|Crime|Thriller,4.0
6,7,Sabrina (1995),Comedy|Romance,3.0
7,8,Tom and Huck (1995),Adventure|Children,3.0
8,9,Sudden Death (1995),Action,3.0
9,10,GoldenEye (1995),Action|Adventure|Thriller,3.5


Отберем фильмы с рейтингом больше 3.5, чтобы не рекомендовать плохие

In [44]:
new_data = new_data[new_data.film_median_rating > 3.5]
new_data

,movieId,title,genres,film_median_rating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,4.0
5,6,Heat (1995),Action|Crime|Thriller,4.0
10,11,"American President, The (1995)",Comedy|Drama|Romance,4.0
13,14,Nixon (1995),Drama,4.0
15,16,Casino (1995),Crime|Drama,4.0
...,...,...,...,...
9713,191005,Gintama (2017),Action|Adventure|Comedy|Sci-Fi,4.5
9716,193571,Silver Spoon (2014),Comedy|Drama,4.0
9717,193573,Love Live! The School Idol Movie (2015),Animation,4.0
9719,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,4.0


In [36]:
# разбиваем каждый жанр по словам: 
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(new_data.genres)
# print(X_train_counts)

In [43]:
df1 = pd.DataFrame(X_train_counts.toarray(), columns=count_vect.get_feature_names())
df1

,action,adventure,animation,children,comedy,crime,documentary,drama,fantasy,fi,...,listed,musical,mystery,no,noir,romance,sci,thriller,war,western
0,0,1,1,1,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3622,1,1,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
3623,0,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3624,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3625,1,0,1,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [47]:
new_data.reset_index(drop=True, inplace=True)
res = pd.concat([new_data, df1], axis=1)
res

,movieId,title,genres,film_median_rating,action,adventure,animation,children,comedy,crime,...,listed,musical,mystery,no,noir,romance,sci,thriller,war,western
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,4.0,0,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,6,Heat (1995),Action|Crime|Thriller,4.0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
2,11,"American President, The (1995)",Comedy|Drama|Romance,4.0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
3,14,Nixon (1995),Drama,4.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,16,Casino (1995),Crime|Drama,4.0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3622,191005,Gintama (2017),Action|Adventure|Comedy|Sci-Fi,4.5,1,1,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
3623,193571,Silver Spoon (2014),Comedy|Drama,4.0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3624,193573,Love Live! The School Idol Movie (2015),Animation,4.0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3625,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,4.0,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [108]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
# print(X_train_tfidf)

In [109]:
from sklearn.neighbors import NearestNeighbors
NN = NearestNeighbors(n_neighbors=10, n_jobs=-1, metric='euclidean') 
NN.fit(X_train_tfidf)

NearestNeighbors(metric='euclidean', n_jobs=-1, n_neighbors=10)

In [110]:
def change_string(s):
    return ' '.join(s.replace(' ', '').replace('-', '').split('|'))

In [113]:
test = change_string("Comedy|Drama|Fantasy")

predict = count_vect.transform([test])
X_tfidf2 = tfidf_transformer.transform(predict)

res = NN.kneighbors(X_tfidf2, return_distance=True)
res

(array([[0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.36867724]]),
 array([[3402, 2888, 1616, 2535, 2817, 3252, 3212,  821, 2048,  862]],
       dtype=int64))

In [114]:
new_data.iloc[res[1][0]]

,movieId,title,genres,film_median_rating
9221,153408,Tears for Sale (2008),Comedy|Drama|Fantasy,4.00
7948,96430,"Odd Life of Timothy Green, The (2012)",Comedy|Drama|Fantasy,5.00
4498,6666,"Discreet Charm of the Bourgeoisie, The (Charme...",Comedy|Drama|Fantasy,4.50
7036,69304,Imagine That (2009),Comedy|Drama|Fantasy,4.00
7764,91784,Girl Walks Into a Bar (2011),Comedy|Drama|Fantasy,4.50
8868,134252,That Munchhausen (1979),Comedy|Drama|Fantasy,4.50
8780,130087,The Cobbler (2015),Comedy|Drama|Fantasy,3.75
2257,2997,Being John Malkovich (1999),Comedy|Drama|Fantasy,4.00
5620,27320,"Nine Lives of Tomas Katz, The (2000)",Comedy|Drama|Fantasy,5.00
2329,3088,Harvey (1950),Comedy|Fantasy,4.00


In [116]:
movies_with_tags = movies.join(tags.set_index('movieId'), on='movieId')

In [117]:
movies_with_tags.head()

,movieId,title,genres,userId,tag,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,1.139046e+09
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,474.0,pixar,1.137207e+09
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,567.0,fun,1.525286e+09
1,2,Jumanji (1995),Adventure|Children|Fantasy,62.0,fantasy,1.528844e+09
1,2,Jumanji (1995),Adventure|Children|Fantasy,62.0,magic board game,1.528844e+09


In [118]:
movies_with_tags.tag.unique()

array(['pixar', 'fun', 'fantasy', ..., 'star wars', 'gintama', 'remaster'],
      dtype=object)

In [119]:
movies_with_tags.tag.unique().shape

(1590,)

In [120]:
movies_with_tags.dropna(inplace=True)

In [121]:
movies_with_tags.title.unique().shape

(1572,)

In [123]:
from tqdm.notebook import tqdm

tag_strings = []
movies = []

for movie, group in tqdm(movies_with_tags.groupby('title')):
    tag_strings.append(' '.join([str(s).replace(' ', '').replace('-', '') for s in group.tag.values]))
    movies.append(movie)

In [147]:
tag_strings[:10]

['artistic Funny humorous inspiring intelligent quirky romance ZooeyDeschanel',
 'lawyers',
 'creepy suspense',
 'Shakespearesortof',
 'dogs remake',
 'Disney',
 'terrorism',
 'court claustrophobic confrontational earnest gooddialogue greatscreenplay gritty Motivational thoughtprovoking',
 'stranded',
 'MarkRuffalo']

In [148]:
count_vect = CountVectorizer()
X_train_counts2 = count_vect.fit_transform(tag_strings)

In [149]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf2 = tfidf_transformer.fit_transform(X_train_counts2)

In [150]:
NN2 = NearestNeighbors(n_neighbors=10, n_jobs=-1, metric='manhattan') 
NN2.fit(X_train_tfidf2)

NearestNeighbors(metric='manhattan', n_jobs=-1, n_neighbors=10)

In [153]:
test2 = change_string('MarkRuffalo')

predict = count_vect.transform([test2])
X_tfidf2 = tfidf_transformer.transform(predict)

res2 = NN2.kneighbors(X_tfidf2, return_distance=True)
res2

(array([[0., 1., 1., 2., 2., 2., 2., 2., 2., 2.]]),
 array([[  9, 822, 661, 949, 950, 953, 951, 955, 954, 947]], dtype=int64))

In [154]:
for i in res2[1][0]:
    print(movies[i])

13 Going on 30 (2004)
Magnolia (1999)
In a Lonely Place (1950)
Net, The (1995)
Network (1976)
Niagara (1953)
Never Been Kissed (1999)
Night and Day (1946)
Nicholas Nickleby (2002)
Neon Genesis Evangelion: Death & Rebirth (Shin seiki Evangelion Gekijô-ban: Shito shinsei) (1997)
